# Selenium을 이용한 동적 크롤링

In [5]:
import requests, time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys        # 기본적으로 불러와야되는.import

In [8]:
from bs4 import BeautifulSoup

url = 'https://sports.news.naver.com/kbaseball/news/index.nhn?isphoto=N'
req = requests.get(url)
soup = BeautifulSoup(req.text, 'html.parser')
## soup   <-- 너무 길어서 생략(결과)
# BeautifulSoup으로 크롤링을 하려고 하였지만 본문은 크롤링이 안됨x

## 야구 > 최신뉴스

In [56]:
driver = webdriver.Chrome('chromedriver')
url = 'https://sports.news.naver.com/kbaseball/news/index.nhn?isphoto=N'
driver.get(url)

In [57]:
# 최신 뉴스 리스트의 맨 마지막 페이지 (3.24일꺼로함)
while True:
    page = driver.find_element_by_id('_pageList')
    try:
        next = page.find_element_by_class_name('next')
        if next:
            next.click()
    except:         # exception은 마지막페이지에 발생
        break;      # break를 걸어서 무한루프를 퇴출, 

In [58]:
page = driver.find_element_by_id('_pageList')
try:
    atags = page.find_elements_by_tag_name('a')
    numPage = int(atags[-1].text)
except:
    numPage = 1
numPage

1

In [59]:
news = driver.find_element_by_id('_newsList')
lis = news.find_elements_by_tag_name('li')
len(lis)
## 3.23실습은 페이지 20이었으나, 3.24일로 복습함

3

In [60]:
for li in lis:
    text = li.find_element_by_class_name('title')
        title = text.find_element_by_tag_name('span').text
    print(title)

'유쾌한' 힐리 "취재진 덕분에 홈런 쳤다, 고마워"... 왜? [★잠실]
[IS 포커스] 수비 채우고, 공격 때리고…기대 이상 '이용규 효과'
이긴 기억 가물가물…올해도 두산은 강팀일까


### 종목 가져오기

In [61]:
menus = driver.find_elements_by_class_name('main_menu_item')
for menu in menus:
    event = menu.find_element_by_tag_name('a').text
    print(event)

스포츠홈
야구
해외야구
축구
해외축구
농구
배구
골프
일반
e스포츠&게임
오늘의 경기
연재
랭킹


In [62]:
menus = driver.find_elements_by_class_name('main_menu_item')
events = []
for menu in menus[1:-3]:
    event = menu.find_element_by_tag_name('a').get_attribute('href')
    events.append(event)
events

['https://sports.news.naver.com/kbaseball/index.nhn',
 'https://sports.news.naver.com/wbaseball/index.nhn',
 'https://sports.news.naver.com/kfootball/index.nhn',
 'https://sports.news.naver.com/wfootball/index.nhn',
 'https://sports.news.naver.com/basketball/index.nhn',
 'https://sports.news.naver.com/volleyball/index.nhn',
 'https://sports.news.naver.com/golf/index.nhn',
 'https://sports.news.naver.com/general/index.nhn',
 'https://sports.news.naver.com/esports/index.nhn']

In [63]:
menus = driver.find_elements_by_class_name('main_menu_item')
events = []
for menu in menus[1:-3]:
    event = menu.find_element_by_tag_name('a').get_attribute('href')
    events.append(event.split('/')[3])
events

['kbaseball',
 'wbaseball',
 'kfootball',
 'wfootball',
 'basketball',
 'volleyball',
 'golf',
 'general',
 'esports']

In [67]:
# 페이지 수

base_url = 'https://sports.news.naver.com/'
sub_url = '/news/index.nhn?isphoto=N'
for event in events:
    print(event, end=': ')
    url = f'{base_url}{event}{sub_url}'
    driver.get(url)
    time.sleep(1)

    while True:
        page = driver.find_element_by_id('_pageList')
        try:
            next = page.find_element_by_class_name('next')
            if next:
                next.click()
        except:
            break;

    page = driver.find_element_by_id('_pageList')
    try:
        atags = page.find_elements_by_tag_name('a')
        numPage = int(atags[-1].text)
    except:
        numPage = 1

    print(numPage)

kbaseball: 1
wbaseball: 1
kfootball: 1
wfootball: 1
basketball: 1
volleyball: 1
golf: 1
general: 1
esports: 1


In [69]:
# 페이지를 돌아다니면서 데이터를 가지고 오는 경우
# 타

title_list = []     # 타이틀을 기록할 빈 리스트 만듦.
for event in events:            
    print(event, end=': ')
    url = f'{base_url}{event}{sub_url}'
    driver.get(url)
    time.sleep(1)

    while True:
        page = driver.find_element_by_id('_pageList')
        try:
            next = page.find_element_by_class_name('next')
            if next:
                next.click()
        except:
            break;

    page = driver.find_element_by_id('_pageList')
    try:
        atags = page.find_elements_by_tag_name('a')
        numPage = int(atags[-1].text)
    except:
        numPage = 1

    print(numPage, end=' - ')

    for page in range(1, numPage+1):
        print(page, end=', ')       # 페이지를 찍어주는.
        url = f'{base_url}{event}{sub_url}&page={page}'
        driver.get(url)
        time.sleep(1)

        news = driver.find_element_by_id('_newsList')
        lis = news.find_elements_by_tag_name('li')
        for li in lis:
            text = li.find_element_by_class_name('title')
            title = text.find_element_by_tag_name('span').text
            title_list.append(title)        # 리스트가 만들어짐.
    print()

kbaseball: 1 - 1, 
wbaseball: 1 - 1, 
kfootball: 1 - 1, 
wfootball: 1 - 1, 
basketball: 1 - 1, 
volleyball: 1 - 1, 
golf: 1 - 1, 
general: 1 - 1, 
esports: 1 - 1, 


In [71]:
# 데이터 title_list가 몇개인지 세어보기
len(title_list) 

43

In [72]:
## sports.txt 기록(저장,생성)

title_str = '\n'.join(title for title in title_list)
with open('sports.txt', 'w', encoding='utf-8') as fp:
    fp.write(title_str)